In [32]:
import glob
import time
import pandas as pd
from tqdm import tqdm

files = glob.glob('../../bitcoin_prediction/data/*/*/*/data.csv', recursive=True)
filter_files = [
    'year_2021/month_4',
    'year_2021/month_5',
    'year_2021/month_6',
]

files = [x for x in files if
               all(y not in x for y in filter_files)]

btc_file = glob.glob('../../bitcoin_prediction/data/BTC-USD.csv', recursive=True)[0]
btc_dataset = pd.read_csv(btc_file, lineterminator='\n').drop(columns=["High", "Low", "Close", "Adj Close"], axis=1)


In [34]:
historical_sentiment = pd.DataFrame(columns=["Date", "Negative", "Neutral", "Positive"])

for file in tqdm(files):
    dataset = pd.read_csv(file, lineterminator='\n').drop(columns=["text", "id"], axis=1)

    dataset["created_at"] = pd.to_datetime(dataset["created_at"])
    dataset["created_at"] = dataset["created_at"].dt.strftime('%Y-%m-%d')

    dataset["sentiment_count"] = dataset.groupby("Sentiment")["Sentiment"].transform('count')

    dataset = dataset.drop_duplicates(subset="Sentiment", keep="last")

    dataset = dataset.pivot('created_at', 'Sentiment').stack(0).reset_index()
    dataset = dataset.rename_axis(None, axis=1).reset_index(drop=True)

    dataset = dataset.drop(columns=["level_1"], axis=0).rename(columns={ 'created_at': 'Date', 0: "Negative", 1: "Neutral", 2: "Positive" })

    final_dataset = pd.concat([final_dataset, dataset], axis=0)

historical_sentiment = final_dataset.sort_values(by='Date')

merged_datasets = pd.merge(final_dataset, btc_dataset, how="outer", on="Date").sort_values(by='Date').drop_duplicates(subset="Date", keep="last")

merged_datasets.to_csv('../../bitcoin_prediction/data/historical_btc_sentiment.csv', index=False)

100%|█████████████████████████████████████████| 243/243 [01:11<00:00,  3.40it/s]
